## Solving bidomain equations on the ventricle with modified FitzHugh-Nagumo model

### Importing required libraries

In [1]:
%cd ../../..
from src.models.dynamics_models import *
from src.models.cell_models import *
from src.utils import *
import ufl

/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-tkiva


### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = heart_ventricle()
x = ufl.SpatialCoordinate(domain)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 50%] Meshing curve 6 (Ellipse)
Info    : [ 60%] Meshing curve 7 (Ellipse)
Info    : [ 80%] Meshing curve 9 (Ellipse)
Info    : [ 90%] Meshing curve 10 (Ellipse)
Info    : [100%] Meshing curve 11 (Line)
Info    : Done meshing 1D (Wall 0.0123245s, CPU 0.012287s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Surface of Revolution, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Surface of Revolution, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.139531s, CPU 0.139678s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 3157 nodes...


In [3]:
class Model(BidomainModel):
    def initial_V_m(self):
        def value(x):
            x_c, y_c, z_c = 0, 0, -7.5
            a, b, c = 1, 1, 1
            return -85 + 35 * np.exp(
                (
                    -(((x[0] - x_c) / a) ** 2)
                    - ((x[1] - y_c) / b) ** 2
                    - ((x[2] - z_c) / c) ** 2
                )
                * np.log(10)
            )

        self.V_m_n.interpolate(value)
        return self.V_m_n

    def conductivity(self):
        # Muscle fibres
        x_c, y_c = 0.2, 1
        self.fibers = ufl.as_vector(
            [
                -(x[1] - y_c) / ufl.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
                (x[0] - x_c) / ufl.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
                0,
            ]
        )
        self.normals = ufl.as_vector([0, 0, 1])

        # Healthy conductivities
        self.M_i = (
            self.SIGMA_IT * ufl.Identity(3)
            + (self.SIGMA_IL - self.SIGMA_IT) * ufl.outer(self.fibers, self.fibers)
            + (self.SIGMA_IN - self.SIGMA_IT) * ufl.outer(self.normals, self.normals)
        )
        self.M_e = (
            self.SIGMA_ET * ufl.Identity(3)
            + (self.SIGMA_EL - self.SIGMA_ET) * ufl.outer(self.fibers, self.fibers)
            + (self.SIGMA_EN - self.SIGMA_ET) * ufl.outer(self.normals, self.normals)
        )

    def ischemia(self):
        x_c, y_c, z_c = -0.7, -1.8, -4.0
        a, b, c = 1.0, 3.0, 0.5

        def reduce(x, reduce_factor: float = 15):
            return 1 + (reduce_factor - 1) * ufl.exp(
                (
                    -(((x[0] - x_c) / a) ** 2)
                    - ((x[1] - y_c) / b) ** 2
                    - ((x[2] - z_c) / c) ** 2
                )
                * ufl.ln(10)
            )

        def value(x, reduce_factor: float = 15):
            return 1 + (reduce_factor - 1) * np.exp(
                (
                    -(((x[0] - x_c) / a) ** 2)
                    - ((x[1] - y_c) / b) ** 2
                    - ((x[2] - z_c) / c) ** 2
                )
                * np.log(10)
            )

        self.M_i = self.M_i / reduce(self.x)
        self.M_e = self.M_e / reduce(self.x)

        fun = fem.Function(self.V1)
        fun.interpolate(value)
        return fun

In [4]:
cell_model = ReparametrizedFitzHughNagumo(domain)
model = Model(domain, cell_model)

In [5]:
plot_function(
    model.initial_V_m(),
    save_to="figures/ventricle/ischemia_MFN_initial_V_m.pdf",
)

Widget(value='<iframe src="http://localhost:35835/index.html?ui=P_0x7f00d45a5910_0&reconnect=auto" class="pyvi…

In [6]:
plot_function(
    model.ischemia(),
    save_to="figures/ventricle/ischemia_MFN_ischemia_location.pdf",
)

Widget(value='<iframe src="http://localhost:35835/index.html?ui=P_0x7f0130cd8550_1&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [ ]:
model.solve(
    T=2000,
    steps=2000,
    save_to="ventricle/ischemia_MFN.mp4",
    checkpoints=[500, 1000, 1500, 2000],
    checkpoint_file="ventricle/ischemia_MFN",
)

### Plotting fiber orientations

In [ ]:
x_c, y_c = 0, 0
plot_vector_field(
    domain,
    lambda x: (
        -(x[1] - y_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        (x[0] - x_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        0,
    ),
    0.02,
    0.4,
    save_to="figures/ventricle/ischemia_MFN_fibers.pdf",
)